# Precipitacion

Para ver el concepto de Precipitacion y Cambio Climático revise la [Introduccion](Introduction.ipynb).
+ Mapas de Precipitacion Total de años recientes de cada departamento.

## Mapas de Precipitacion Total

In [1]:
#Se descargan las librerias necesarias 
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import cartopy as cart
import scipy.stats #Metodos estadisticos
import matplotlib.colors as colors #Set Colors
import wradlib.ipol as ipol #Metodo de Interpolación

class Mapa:  
    # El metodo inicial o constructor
    def __init__(self, departamento, year, mes, tipo):
     
        # Instance Variable    
        self.departamento = departamento
        self.year = year
        self.mes = mes
        self.tipo = tipo
        self.normal = '1981-2010' 
        
    # El metodo para contruir mapas     
    def get_mapa(self, ax, periodo):
        
        #Se define la extensión del mapa
        if str(self.departamento) == 'Santander':
            lllat, urlat, lllon, urlon = 5.5, 8.5, -75, -72   
        elif str(self.departamento) == 'Magdalena':
            lllat, urlat, lllon, urlon = 8.7, 11.5, -75.55, -73
    
        #Se establece instancia de Cartopy    
        ax.set_extent([lllon, urlon, lllat, urlat], crs=cart.crs.PlateCarree())
        ax.add_feature(cart.feature.OCEAN, zorder=100, edgecolor='k')#, facecolor='white')
    
        #Se dibujan los departamentos de Colombia
        fname = r".\DIVISION_COLOMBIA\MGN_DPTO_POLITICO.shp"
        adm1_shapes = list(cart.io.shapereader.Reader(fname).geometries())
        ax.add_geometries(adm1_shapes, cart.crs.PlateCarree(), edgecolor='black', facecolor='none', alpha=0.5)

        #Se cargan los datos de las estaciones    
        df = pd.read_csv(str(r".\Datos\Precipitacion\Precipitacion_"+str(periodo)+".csv"))

        #Se seleccionan las estaciones que se ubiquen en el area del mapa 
        data_magdalena = df.loc[(df['LONGITUD'] < urlon) & (df['LONGITUD'] > lllon)
                  & (df['LATITUD'] < urlat) & (df['LATITUD'] > lllat)]

        #Se remueven filas de dataframe que tengan valores nan para el mes especifico
        data_magdalena = data_magdalena.dropna(subset=[str(self.mes)])

        #Se extrae los valores de coordenadas, altura y valor de cada estacion
        longitud = data_magdalena['LONGITUD'].tolist()
        latitud = data_magdalena['LATITUD'].tolist()
        elevacion = data_magdalena['ELEVACION'].tolist()
        valor = data_magdalena[str(self.mes)].tolist()
        src = np.vstack( (np.array(longitud), np.array(latitud)) ).transpose()

        #Se crea una grilla 100x100
        numcols, numrows = 350, 350
        xtrg = np.linspace(lllon, urlon, numcols)
        ytrg = np.linspace(lllat, urlat, numrows)
        trg = np.meshgrid(xtrg, ytrg)
        trg = np.vstack( (trg[0].ravel(), trg[1].ravel()) ).T

        #Se aplica el método de interpolación OK (Ordinary Kriging)
        idw = ipol.OrdinaryKriging(src, trg)
        zi = idw(np.array(valor))
  
        if str(periodo) == "1981-2010":
            self.Normal, self.x, self.y = zi, xtrg, ytrg
        elif str(periodo) != "1981-2010" and str(self.tipo) == "Anomalia":
            return zi, ax
        
        #Se dibuja los valores en el mapa
        cmap = plt.get_cmap('hsv',16)
        con = plt.pcolormesh(xtrg, ytrg, zi.reshape( (numcols, numrows) ), transform=cart.crs.PlateCarree(), alpha=0.6, cmap = cmap, vmin = 0, vmax = 800)

        #Se dibuja la grafica de colores
        cbar = plt.colorbar(con, orientation='horizontal',fraction=.042, pad=0.03)
        
    def plot_mapas(self):
        # Se define el tamaño del cuadro 
        plt.figure(figsize=(12.4, 9.4))

        # Tabla 1x3 y dibujaremos en la celda 1
        ax = plt.subplot(1, 2, 1, projection=cart.crs.PlateCarree())
        ax.set_title('Normal Climatologica Precipitacion (1981-2010)',fontdict = {'fontsize':13})
        self.get_mapa(ax,self.normal)
        normal, x, y = self.Normal, self.x, self.y

        # Tabla 1x3 y dibujaremos en la celda 2
        ax = plt.subplot(1, 2, 2, projection=cart.crs.PlateCarree()) 
        ax.set_title('Precipitacion Total '+str(self.year),fontdict = {'fontsize':13})    
    
        #Se obtiene el mapa de Índice de Precipitacion
        if str(self.tipo) == 'Anomalia':
            actual, ax = self.get_mapa(ax,self.year) 
            anomalia = np.divide(actual,normal)*100
            cmap = colors.ListedColormap(["#ffaa01","#ffd380","#fffbc3","#ffffff","#bee9fe","#72b3ff","#0171ff"]) 
            levels = [0,30, 60, 90, 110, 140, 170, 500]
            norm = colors.BoundaryNorm(levels, cmap.N)
            con = plt.pcolormesh(x,y,anomalia.reshape( (350, 350) ), transform=cart.crs.PlateCarree(), alpha=0.6, cmap = cmap, norm = norm)
            cbar = plt.colorbar(con, orientation='horizontal',fraction=.042, pad=0.03)
            ax.set_title('Índice de Precipitacion '+str(self.year),fontdict = {'fontsize':13})
        else:
            self.get_mapa(ax,self.year)
 
        # Dibujamos el conjunto
        plt.show()   

In [5]:
import ipywidgets as widgets
import time

a = widgets.Dropdown(options=[('Enero', 'ENE'), ('Febrero', 'FEB'), ('Marzo', 'MAR'), ('Abril', 'ABR'), ('Mayo', 'MAY'),
                             ('Junio', 'JUN'), ('Julio', 'JUL'), ('Agosto', 'AGO'), ('Septiembre', 'SEP'),
                             ('Octubre', 'OCT'), ('Noviembre', 'NOV'), ('Diciembre', 'DIC')], description='Mes:')
b = widgets.Dropdown(options=[2011,2012,2013,2014,2015,2016,2017,2018,2019,2020], description='Año:')
c = widgets.Dropdown(options=['Magdalena', 'Santander'], description='Dep:')
d = widgets.Dropdown(options=[('Anomalía', 'Anomalia'), ('Valor Absoluto', 'Absoluto')], description='Tipo:')
ui = widgets.HBox([a, b, c, d])
def f(a, b, c, d):
    start_time = time.time()
    m = Mapa(c,b,a,d)
    m.plot_mapas()
    print("--- %s seconds ---" % (time.time() - start_time))

out = widgets.interactive_output(f, {'a': a, 'b': b, 'c': c, 'd': d})

display(ui, out)

Output()

In [3]:
#####################################################################
# Se programa para que se esconda el código de este Jupyter Notebook
#####################################################################

In [4]:
from IPython.display import HTML
HTML('''<script>
code_show=true; 
function code_toggle() {
 if (code_show){
 $('div.input').hide();
 } else {
 $('div.input').show();
 }
 code_show = !code_show
} 
$( document ).ready(code_toggle);
</script>
El código de este cuaderno IPython está oculto de forma predeterminada para facilitar la lectura.
Para activar / desactivar el código sin formato, haga clic <a href="javascript:code_toggle()">aqui</a>.''')